In [2]:
import torch
import json
from tqdm import tqdm
from itertools import product as p
import numpy as np

# Load fake activations

In [3]:
class A:
    pass
self = A()

In [4]:
self.num_neurons_d = {} # {fname, int}
self.representations_d = {} # {fname, tensor}
f1, f2, f3 = "foo", "bar", "baz"
representation_files = [f1, f2, f3]

In [5]:
# initialize `num_neurons_d`, `representations_d` with fake data
n1, n2, n3 = 100, 80, 70
nword = 1000
t1 = torch.randn(nword, n1)
t2 = torch.randn(nword, n2)
t3 = torch.randn(nword, n3)
self.num_neurons_d[f1] = n1
self.num_neurons_d[f2] = n2
self.num_neurons_d[f3] = n3
self.representations_d[f1] = t1
self.representations_d[f2] = t2
self.representations_d[f3] = t3

In [6]:
self.normalize_dimensions = True

# Compute Correlations

In [7]:
def compute_correlations(self):
    """
    Set `self.similarities`. 
    """
    # Normalize
    if self.normalize_dimensions:
        for network in tqdm(self.representations_d, desc='mu, sigma'):
            # TODO: might not need to normalize, only center
            t = self.representations_d[network]
            means = t.mean(0, keepdim=True)
            stdevs = t.std(0, keepdim=True)

            self.representations_d[network] = (t - means) / stdevs

    # Set `self.similarities`
    # {network: {other: cka_similarity}}
    self.similarities = {network: {} for network in self.representations_d}
    for network, other_network in tqdm(p(self.representations_d,
                                         self.representations_d), desc='cka',
                                       total=len(self.representations_d)**2):

        if network == other_network:
            continue

        if other_network in self.similarities[network].keys(): # TO DO: optimize?
            continue

        X = self.representations_d[network]
        Y = self.representations_d[other_network]

        XtX_F = torch.norm(torch.mm(X.t(), X), p='fro').item()
        YtY_F = torch.norm(torch.mm(Y.t(), Y), p='fro').item()
        YtX_F = torch.norm(torch.mm(Y.t(), X), p='fro').item()

        # eq 5 in paper
        self.similarities[network][other_network] = YtX_F**2 / (XtX_F*YtY_F)

In [8]:
compute_correlations(self)

cka: 100%|█| 9/9 [00:00<00:00, 1341.56it/s]


# Build function

In [9]:
output_file = "temp"

In [11]:
torch.save(self.similarities, output_file)

In [12]:
torch.load(output_file)

{'foo': {'bar': 0.08240745869842656, 'baz': 0.07658038898416103},
 'bar': {'foo': 0.0824072311273204, 'baz': 0.06845917054715424},
 'baz': {'foo': 0.07658026112880613, 'bar': 0.06845918481026368}}